# 🎭 Face Parsing: Semantic Segmentation

<div style="display:flex; flex-wrap:wrap; align-items:center;">
  <a style="margin-right:10px; margin-bottom:6px;" href="https://pepy.tech/projects/uniface"><img alt="PyPI Downloads" src="https://static.pepy.tech/badge/uniface"></a>
  <a style="margin-right:10px; margin-bottom:6px;" href="https://pypi.org/project/uniface/"><img alt="PyPI Version" src="https://img.shields.io/pypi/v/uniface.svg"></a>
  <a style="margin-right:10px; margin-bottom:6px;" href="https://opensource.org/licenses/MIT"><img alt="License" src="https://img.shields.io/badge/License-MIT-blue.svg"></a>
  <a style="margin-bottom:6px;" href="https://github.com/yakhyo/uniface"><img alt="GitHub Stars" src="https://img.shields.io/github/stars/yakhyo/uniface.svg?style=social"></a>
</div>

**UniFace** is a lightweight, production-ready, all-in-one face analysis library built on ONNX Runtime.

🔗 **GitHub**: [github.com/yakhyo/uniface](https://github.com/yakhyo/uniface) | 📚 **Docs**: [yakhyo.github.io/uniface](https://yakhyo.github.io/uniface)

---

## 📖 Overview

This notebook demonstrates **face parsing** - semantic segmentation of facial components:

- ✅ Segment faces into 19 different components (skin, eyes, hair, etc.)
- ✅ Visualize parsing results with color overlays
- ✅ Extract specific facial components using masks

## 1️⃣ Installation

In [ ]:
%pip install -q uniface

import os
import urllib.request

os.makedirs('assets/test_images', exist_ok=True)

BASE_URL = "https://raw.githubusercontent.com/yakhyo/uniface/main/assets"
images = ["test_images/image0.jpg", "test_images/image1.jpg", "test_images/image2.jpg",
          "test_images/image3.jpg", "test_images/image4.jpg"]

for img in images:
    if not os.path.exists(f'assets/{img}'):
        urllib.request.urlretrieve(f"{BASE_URL}/{img}", f"assets/{img}")
        print(f"✓ Downloaded {img}")

## 2️⃣ Import Libraries

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

import uniface
from uniface.parsing import BiSeNet
from uniface.constants import ParsingWeights
from uniface.visualization import vis_parsing_maps

print(f"UniFace version: {uniface.__version__}")

## 3️⃣ Initialize BiSeNet Parser

In [ ]:
parser = BiSeNet(model_name=ParsingWeights.RESNET34)

## 4️⃣ Parsing Classes Reference

The BiSeNet model segments faces into **19 different classes**:

| ID | Component | ID | Component |
|:--:|-----------|:--:|-----------|
| 0 | Background | 10 | Nose |
| 1 | Skin | 11 | Mouth |
| 2 | Left Eyebrow | 12 | Upper Lip |
| 3 | Right Eyebrow | 13 | Lower Lip |
| 4 | Left Eye | 14 | Neck |
| 5 | Right Eye | 15 | Neck Lace |
| 6 | Eye Glasses | 16 | Cloth |
| 7 | Left Ear | 17 | Hair |
| 8 | Right Ear | 18 | Hat |
| 9 | Ear Ring | | |

## 5️⃣ Process Multiple Face Images

The test images are already cropped faces, so we can directly parse them.

In [ ]:
test_images_dir = Path('assets/test_images')
test_images = sorted(test_images_dir.glob('*.jpg'))

original_images = []
parsed_images = []

for image_path in test_images:
    print(f"Processing: {image_path.name}")
    image = cv2.imread(str(image_path))
    mask = parser.parse(image)
    unique_classes = len(set(mask.flatten()))
    print(f'   ✓ Found {unique_classes} unique classes')

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    vis_result = vis_parsing_maps(image_rgb, mask, save_image=False)

    original_images.append(image_rgb)
    parsed_images.append(vis_result)

print(f"\n✓ Processed {len(test_images)} images")

## 6️⃣ Visualize Results

**Row 1**: Original face images  
**Row 2**: Parsed images with color overlay

In [ ]:
num_images = len(original_images)
fig, axes = plt.subplots(2, num_images, figsize=(4 * num_images, 8))

if num_images == 1:
    axes = axes.reshape(-1, 1)

for i in range(num_images):
    axes[0, i].imshow(original_images[i])
    axes[0, i].set_title(f'Original {i+1}', fontsize=12)
    axes[0, i].axis('off')

    axes[1, i].imshow(parsed_images[i])
    axes[1, i].set_title(f'Parsed {i+1}', fontsize=12)
    axes[1, i].axis('off')

plt.tight_layout()
plt.show()

## 7️⃣ Detailed Parsing View

Let's parse a single face and show the segmentation mask in detail.

In [ ]:
image_path = 'assets/test_images/image1.jpg'
image = cv2.imread(image_path)
mask = parser.parse(image)

image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
vis_result = vis_parsing_maps(image_rgb, mask, save_image=False)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(image_rgb)
axes[0].set_title('Original Face', fontsize=14)
axes[0].axis('off')

axes[1].imshow(mask, cmap='tab20')
axes[1].set_title('Segmentation Mask', fontsize=14)
axes[1].axis('off')

axes[2].imshow(vis_result)
axes[2].set_title('Overlay Visualization', fontsize=14)
axes[2].axis('off')

plt.tight_layout()
plt.show()

print(f"📊 Mask shape: {mask.shape}")
print(f"📊 Unique classes: {np.unique(mask)}")
print(f"📊 Number of classes found: {len(np.unique(mask))}")

## 8️⃣ Extract Specific Facial Components

Use the mask to extract individual facial components.

In [ ]:
image_path = 'assets/test_images/image0.jpg'
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
mask = parser.parse(image)

components_to_extract = {'Skin': 1, 'Hair': 17, 'Nose': 10, 'Lips': [12, 13]}

fig, axes = plt.subplots(1, len(components_to_extract) + 1, figsize=(20, 4))

axes[0].imshow(image_rgb)
axes[0].set_title('Original', fontsize=12)
axes[0].axis('off')

for idx, (name, class_ids) in enumerate(components_to_extract.items(), 1):
    if isinstance(class_ids, list):
        component_mask = np.zeros_like(mask, dtype=np.uint8)
        for class_id in class_ids:
            component_mask |= (mask == class_id).astype(np.uint8)
    else:
        component_mask = (mask == class_ids).astype(np.uint8)

    extracted = image_rgb.copy()
    extracted[component_mask == 0] = 0

    axes[idx].imshow(extracted)
    axes[idx].set_title(name, fontsize=12)
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

---

## 📝 Summary

| Feature | Description |
|---------|-------------|
| **Model** | BiSeNet (ResNet18 or ResNet34 backbone) |
| **Classes** | 19 semantic classes |
| **Input** | Face image (cropped or full) |
| **Output** | Segmentation mask (H×W) with class IDs |

### Use Cases

- 🎨 Virtual makeup application
- 💇 Hair color change / style preview
- 🔬 Skin analysis and retouching
- 🎭 Face editing and composition

---

## 🔗 Next Steps

- **Face Anonymization**: Blur faces for privacy → [07_face_anonymization.ipynb](./07_face_anonymization.ipynb)
- **Gaze Estimation**: Predict eye gaze direction → [08_gaze_estimation.ipynb](./08_gaze_estimation.ipynb)
- **Full Documentation**: [yakhyo.github.io/uniface](https://yakhyo.github.io/uniface)